One of the challenges for automated stem mapping is that the paremeters for stem mapping require existing knowledge about the plot, such as how many trees could reasonably exist and their sizes. Otherwise the search space is so large that automatic detections are inpractical due to low precision. This notebook demonstrates how we can limit the search space into something more reasonable for automatic stem mapping.

In [ ]:
# load the necessary libraries

import matplotlib.pyplot as plt
import numpy as np

from math import floor

from PIL import Image

from scipy import ndimage

from skimage.draw import circle_perimeter, line
from skimage import filters, measure

This 5x5 m plot has several trees we want to detect, as well as noise from a fallen log and several shrubs.

In [ ]:
input_img = Image.open('data/cluster_segmentation/cross_section.jpg')
input_img = np.array(input_img)

# multiplying the intesity values 100 makes it easier to see
plt.imshow(input_img*100)

The first thing to note is that while the total area is 5x5 meters, much of that area is empty space. A distance transform can find the natural boundaries between groups of pixels.

In [ ]:
dist = ndimage.distance_transform_edt(np.logical_not(input_img))

plt.imshow(dist)

A ridge following algorithm forms connects segments of high distance pixels. The Meijering algorithm was originally developed for identifying neurites from medical images. Black ridges is set to false because we want to segment areas with high distances from other pixels. The thickness of the segment corresponds to the distaces between areas.

In [ ]:
ridges = filters.meijering(dist, black_ridges=False)
plt.imshow(ridges)

These ridges separate the image into clusters of pixels. Connected component assigns each pixel into one of those clusters. This 5x5m image is now divided into 27 smaller images.

In [ ]:
dimensions = ridges.shape
blobs = ridges.copy()
for i in range(0, dimensions[0]):
    for j in range(0, dimensions[1]):
        pixel = ridges[i][j]
        if pixel < 0.5:
            blobs[i][j] = 1
        else:
            blobs[i][j] = 0
blobs_labels = measure.label(blobs, connectivity=1)

plt.imshow(blobs_labels)

These clusters can be used to mask the original image. I defined some convenience functions below that I will use to crop the images to focus on just the cluster.

In [ ]:
# here are three convenience functions to clean up the clusters

def trim2DArray(input_arr, threshold=0):  
    print(input_arr.shape)
    ul_x, ul_y = calculateTrimOffsetForward(input_arr, threshold)
    lr_x, lr_y = calculateTrimOffsetBackward(input_arr, threshold)

    output_arr = input_arr[ul_y:lr_y, ul_x:lr_x]

    print(output_arr.shape)

    return (output_arr)

def calculateTrimOffsetForward(input_arr, threshold=0):
    ul_x, ul_y = 0, 0

    row_sum = np.sum(input_arr, axis=1)
    col_sum = np.sum(input_arr, axis=0)

    for i in range(0, len(col_sum)):
        if col_sum[i] > threshold:
            ul_x = i
            break
    for j in range(0, len(row_sum)):     
        if row_sum[j] > threshold:
            ul_y = j
            break
    return (ul_x, ul_y)

def calculateTrimOffsetBackward(input_arr, threshold=0):
    y,x = input_arr.shape
    lr_x, lr_y = 0, 0

    row_sum = np.sum(np.flip(input_arr), axis=1)
    col_sum = np.sum(np.flip(input_arr), axis=0)

    for i in range(0, len(col_sum)):
        if col_sum[i] > threshold:
            lr_x = i
            break
    for j in range(0, len(row_sum)):     
        if row_sum[j] > threshold:
            lr_y = j
            break
    return (x-lr_x, y-lr_y)

Now we can investigate each of the clusters individually. I picked out some clusters to see below. The fact that this cluster has a roughly square bounding box is good a sign. We're looking for circles and circles are contained by squares. 

In [ ]:
cluster_6 = np.where(blobs_labels == 6, True, False)

trimmed_6 = trim2DArray(cluster_6)

plt.imshow(trimmed_6)

Now we can use that cluster to mask the original image and see what caused it. This is one of the best case scenarios for a cluster because there is one tree and practically no noise. It will be easy to search this image for circles.

In [ ]:
ul_x, ul_y = calculateTrimOffsetForward(cluster_6)
lr_x, lr_y = calculateTrimOffsetBackward(cluster_6)

one_tree = cluster_6*input_img

trimmed_tree = one_tree[ul_y:lr_y,ul_x:lr_x]

plt.imshow(trimmed_tree*10)

Now let's take a look at a cluster of noise. This very rectangular bounding box is not a good sign, because the pixels here are probably not a circle.

In [ ]:
cluster_11 = np.where(blobs_labels == 11, True, False)

trimmed_11 = trim2DArray(cluster_11)

plt.imshow(trimmed_11)

After applying this cluster as a mask, we can see that this is the log from the lower left corner of the original image.

In [ ]:
ul_x, ul_y = calculateTrimOffsetForward(cluster_11)
lr_x, lr_y = calculateTrimOffsetBackward(cluster_11)

noise = cluster_11*input_img

trimmed_noise = noise[ul_y:lr_y,ul_x:lr_x]

plt.imshow(trimmed_noise*10)

These clusters are important for many reasons.
* The maximum size of a circle in a cluster is half the shortest side of the cluster's bounding box
* The area of the cluster limits the maximum number of trees that can be found
* Clusters that have a side smaller than min_r/2 can be excluded
* Limiting the search space of Hough transforms reduces false positives
* Boundaries between clusters are areas with no pixels

For the trimmed tree, this means that we can limit the search space to circles with a radius between min_r and 0.57 m.

In [ ]:
max_cir = trimmed_tree.copy()

l, w = max_cir.shape
c_x = floor(w/2)
c_y = floor(l/2)

if l > w:
    print("the shortest side is", w)
    r = line(c_y, c_x, c_y, 0)
    sub_y, sub_x = circle_perimeter(c_y, c_x, c_x-1)
else:
    print("the shortest side is", l)
    r = line(c_y, c_x, 0, c_x)
    sub_y, sub_x = circle_perimeter(c_y, c_x, c_y-1)



max_cir[r] = 255
max_cir[sub_y, sub_x] = 255

plt.imshow(max_cir)

For the log cluster, the maximum radius is particularly useful. Even though the image is very tall, we only need to look for circles between min_r and 0.34 meters.

In [ ]:
max_cir = trimmed_noise.copy()

l, w = max_cir.shape
c_x = floor(w/2)
c_y = floor(l/2)

if l > w:
    print("the shortest side is", w)
    r = line(c_y, c_x, c_y, 0)
    sub_y, sub_x = circle_perimeter(c_y, c_x, c_x)
else:
    print("the shortest side is", l)
    r = line(c_y, c_x, 0, c_x)
    sub_y, sub_x = circle_perimeter(c_y, c_x, c_y)



max_cir[r] = 255
max_cir[sub_y, sub_x] = 255

plt.imshow(max_cir)